# APIs

APIs são mecanismos que permitem que dois componentes de software se comuniquem usando um conjunto de definições e protocolos. Por exemplo, o sistema de software do instituto meteorológico contém dados meteorológicos diários. O aplicativo meteorológico em seu telefone “fala” com este sistema por meio de APIs e mostra atualizações meteorológicas diárias no telefone.

API significa Application Programming Interface (Interface de Programação de Aplicação). No contexto de APIs, a palavra Aplicação refere-se a qualquer software com uma função distinta. A interface pode ser pensada como um contrato de serviço entre duas aplicações. Esse contrato define como as duas se comunicam usando solicitações e respostas. A documentação de suas respectivas APIs contém informações sobre como os desenvolvedores devem estruturar essas solicitações e respostas.

![image.png](images/api.jpeg)

`client`: O cliente

`api`: O garçom

`o serviço (banco de dados, web application, twitter, spotify)`: O pizzaiolo

# Web APIs

## Alguns Exemplos

### API endpoints

**NASA API**: http://api.open-notify.org/
- http://api.open-notify.org/iss-now.json
    - Documentation: http://open-notify.org/Open-Notify-API/ISS-Location-Now/

**Meme API**

- http://apimeme.com/?ref=apilist.fun
- http://apimeme.com/meme?meme=Doge&top=Analista&bottom=de_Dados
    

**Public APIs**

- https://github.com/public-apis/public-apis

## Acessando APIs via Requests

`Requests` é uma biblioteca Python que nos permite interagir com servidores [HTTP](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol). Essa biblioteca pode ser utilizada para baixar páginas, arquivos e **interagir com APIs**.



[Documentação - requests](https://requests.readthedocs.io)

In [ ]:
!pip3 install requests

In [1]:
import requests, time

### Utilizando `requests`

Vamos começar acessando uma API simples que nos permite extrair, a cada 5s, a posição da [ISS](https://en.wikipedia.org/wiki/International_Space_Station).

O primeiro passo é descobrir o `endpoint` desta API. O `endpoint` é o URL através do qual acessamos uma API!

In [2]:
endpoint = 'http://api.open-notify.org/iss-now.json'

Com nosso URL em mãos, podemos utilizar a biblioteca `requests` para extrair a informação da API:

In [3]:
response = requests.get(endpoint)

O que é o objeto que guardamos na variável `response`?

In [4]:
response

<Response [200]>

O objeto é do tipo `response` e ele contém o código HTTP associado ao nosso request:

![image.png](images/http_status_codes.jpeg)

Com este código podemos saber se a nossa comunicação com o servidor de certo ou não (e porque não deu certo).

https://www.tutorialspoint.com/http/http_status_codes.htm

Para extrair a informação que a API nos retornou (no caso de um sucesso), podemos utilizar o método `.json()`. 

In [5]:
response.json()

{'message': 'success',
 'timestamp': 1660689925,
 'iss_position': {'latitude': '6.2442', 'longitude': '-96.0601'}}

Como nosso *request* teve sucesso (status 200), o método `.json()` nos retorna o resultado como um dicionário ou uma lista:

In [6]:
resultado = response.json()
type(resultado)

dict

#### Erros de HTTP

Um problema comum ao construirmos scripts que acessam API é a tratativa de erros HTTP. Caso a API não retorne o que esperamos as etapas subsequentes de nosso programa podem falhar. As tratativas de erro HTTP são fundamentais em qualquer programa que tenha extraia informações (seja de uma API, seja via web-scrapping).

Vamos simular um erro tentando acessar um endpoint inexistente:

In [7]:
response = requests.get('http://api.open-notify.org/coisas_aleatória.json')
response

<Response [404]>

Através do objeto `Response` já vemos que o status da execução não é de sucesso: 404 indica um **Client Error**. Podemos utilizar o atributo `.status_code` deste objeto para extrair o código de status diretamente:

In [8]:
response.status_code

404

Poderíamos tratar os diferentes códigos de status através de condicionais. No entanto, como são muitos códigos diferentes e esta é uma tarefa muito comum, o objeto `Response` contém um método que realiza essa tratativa: `.raise_for_status()`

In [9]:
response.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: http://api.open-notify.org/coisas_aleat%C3%B3ria.json

O método `.raise_for_status()` *levanta* um erro toda vez que um código que não seja sucesso é retornado. Sendo assim, podemos utilizar um `try:` e `except:` para tratar este `HTTPError`!

In [10]:
from requests import HTTPError

try:
    response = requests.get('http://api.open-notify.org/coisas_aleatória.json')
    response.raise_for_status()
    resultado = response.json()
except HTTPError as e:
    print(f'Falha na comunicação com o servidor:\n {e}')
    resultado = None

print(resultado)

Falha na comunicação com o servidor:
 404 Client Error: Not Found for url: http://api.open-notify.org/coisas_aleat%C3%B3ria.json
None


Vamos comparar o resultado do bloco acima utilizando o *endpoint* real da API:

In [11]:
try:
    response = requests.get('http://api.open-notify.org/iss-now.json')
    response.raise_for_status()
    resultado = response.json()
except HTTPError as e:
    print(f'Falha na comunicação com o servidor:\n {e}')
    resultado = None

print(resultado)

{'message': 'success', 'timestamp': 1660690453, 'iss_position': {'latitude': '-20.2237', 'longitude': '-76.3621'}}


In [13]:
response.json()

{'message': 'success',
 'timestamp': 1660690453,
 'iss_position': {'latitude': '-20.2237', 'longitude': '-76.3621'}}

## Utilizando APIs I

Até agora vimos como podemos acessar, extrair informações e tratar erros de uma API através de métodos e funções fundamentais. Vamos utilizar este conhecimento para construir um `DataFrame` com as posições da ISS.

In [15]:
from datetime import datetime
import pandas as pd

response = requests.get('http://api.open-notify.org/iss-now.json')
results = response.json()

A primeira etapa de qualquer construção envolvendo uma ou mais APIs é a exploração do objeto retornado.

In [16]:
type(results)

dict

A API da ISS retorna um dicionário: vamos analisar como este dicionário está estruturado:

In [17]:
results.keys()

dict_keys(['message', 'timestamp', 'iss_position'])

Vamos investigar como as informações estão estruturadas dentro destas chaves:

In [18]:
for chave in results.keys():
    print(f"{chave}: {type(results[chave])}")

message: <class 'str'>
timestamp: <class 'int'>
iss_position: <class 'dict'>


As chaves `message` e `timestamp` são tipos simples (`str` e `int` respectivamente). Já a chave `iss_position` é outro dicionário:

In [22]:
float(results['iss_position']['longitude'])

-62.1917

O resultado desta API é muito simples - temos 3 chaves, sendo que 2 são de interesse para nós:

1. `iss_position` contém duas chaves, `latitude` e `longitude`, contendo a posição da ISS;
1. `timestamp` contém o **timestamp** associado a esta posição.

Primeiro, vamos utilizar a função `fromtimestamp()` do módulo `datetime` para converter nosso `timestamp` em um objeto `DateTime`:

In [23]:
results['timestamp']

1660690755

In [24]:
data_hora = datetime.fromtimestamp(results['timestamp'])
print(data_hora)

2022-08-16 19:59:15


Agora que vimos como transformar este resultado, podemos construir um loop para criar uma tabela com a posição da ISS a cada 10s, por 1 minuto:

In [33]:
lat = []
long = []
data_hora = []

for i in range(6):
    response = requests.get('http://api.open-notify.org/iss-now.json')
    results = response.json()

    lat.append(float(results['iss_position']['latitude']))
    long.append(float(results['iss_position']['longitude']))
    data_hora.append(datetime.fromtimestamp(results['timestamp']))

    time.sleep(5)

In [34]:
data_hora

[datetime.datetime(2022, 8, 16, 20, 11, 43),
 datetime.datetime(2022, 8, 16, 20, 11, 49),
 datetime.datetime(2022, 8, 16, 20, 11, 54),
 datetime.datetime(2022, 8, 16, 20, 11, 59),
 datetime.datetime(2022, 8, 16, 20, 12, 5),
 datetime.datetime(2022, 8, 16, 20, 12, 10)]

Agora vamos construir nosso `DataFrame` a partir das listas que criamos em nosso loop:

In [27]:
tb_iss = pd.DataFrame({'lat': lat, 'long':long, 'time':data_hora})

In [28]:
tb_iss

,lat,long,time
0,-43.2498,-47.4471,2022-08-16 20:03:12
1,-43.5853,-46.7290,2022-08-16 20:03:22
2,-43.9320,-45.9659,2022-08-16 20:03:32
3,-44.2730,-45.1935,2022-08-16 20:03:43
4,-44.6081,-44.4116,2022-08-16 20:03:53
5,-44.9216,-43.6581,2022-08-16 20:04:03


In [29]:
tb_iss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   lat     6 non-null      float64       
 1   long    6 non-null      float64       
 2   time    6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 272.0 bytes


In [30]:
lat = []
long = []
data_hora = []
i = 6
while i >= 0:
    response = requests.get('http://api.open-notify.org/iss-now.json')
    results = response.json()

    lat.append(float(results['iss_position']['latitude']))
    long.append(float(results['iss_position']['longitude']))
    data_hora.append(datetime.fromtimestamp(results['timestamp']))
    i = i - 1
    time.sleep(10)

KeyboardInterrupt: 

## Utilizando APIs II

A API da ISS é extremamente rudimentar: ela não possui nem autenticação nem parâmetros, simplesmente retorna a posição atual da ISS. Muitas APIs são mais complexas do que isso, necessitando de autenticação e possibilitando a utilização de parâmetros:

1. **Autenticação**: muitas APIs precisam que o seu programa se autentique, ou seja, se identifique. Muitas vezes isso é feito através do site que disponibiliza a API e requer a criação de um usuário;
1. **Parâmetros**: parâmetros são informações que passamos para a API utilizadas para determinar que informação será retornada.

No próximo exemplo utilizaremos a [WeatherAPI](https://www.weatherapi.com). Esta API retorna informações climáticas para um dado local. Ela é autenticada (através de um *token*) e necessita de uma parametrização (a latitude e longitude do local para o qual queremos extrair as informações).

Primeiro vamos criar especificar todos os parâmetros necessários para a consulta à esta API:

In [36]:
endpoint = 'http://api.weatherapi.com/v1/current.json'
token = 'bfef1b47e3b8462199b191725221801'

header = {'key' : token}
query = {'q' : '-21.996160,-47.426420'}

Utilizamos dois dicionários para especificar tanto a autenticação quanto a parametrização de nosso request:

1. `header`: é um dicionário que contém apenas uma chave, `key`, na qual guardamos nosso token identificador;
1. `query`: é um dicionário que contém apenas uma chave, `q`, na qual guardamos um string de latitude e longitude.

O nome das chaves foi determinado lendo a documentação da API: https://www.weatherapi.com/docs/.

Agora vamos utilizar nossos dicionários e a função `request()` para extrair as informações metereológicas da latitude **-21,996160** e longitude **-47,426420**

In [37]:
response = requests.get(endpoint, headers=header, params=query)
results = response.json()

Utilizamos o argumento `headers =` para passar ao `request()` nossos parâmetros de autenticação, e o argumento `params =` para passar os parâmetros de busca.

In [39]:
results

{'location': {'name': 'Piracununga',
  'region': 'Sao Paulo',
  'country': 'Brazil',
  'lat': -22.0,
  'lon': -47.43,
  'tz_id': 'America/Sao_Paulo',
  'localtime_epoch': 1660692522,
  'localtime': '2022-08-16 20:28'},
 'current': {'last_updated_epoch': 1660691700,
  'last_updated': '2022-08-16 20:15',
  'temp_c': 21.0,
  'temp_f': 69.8,
  'is_day': 0,
  'condition': {'text': 'Clear',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
   'code': 1000},
  'wind_mph': 3.8,
  'wind_kph': 6.1,
  'wind_degree': 300,
  'wind_dir': 'WNW',
  'pressure_mb': 1013.0,
  'pressure_in': 29.91,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 56,
  'cloud': 0,
  'feelslike_c': 21.0,
  'feelslike_f': 69.8,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 8.9,
  'gust_kph': 14.4}}

In [41]:
results['current']

{'last_updated_epoch': 1660691700,
 'last_updated': '2022-08-16 20:15',
 'temp_c': 21.0,
 'temp_f': 69.8,
 'is_day': 0,
 'condition': {'text': 'Clear',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
  'code': 1000},
 'wind_mph': 3.8,
 'wind_kph': 6.1,
 'wind_degree': 300,
 'wind_dir': 'WNW',
 'pressure_mb': 1013.0,
 'pressure_in': 29.91,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 56,
 'cloud': 0,
 'feelslike_c': 21.0,
 'feelslike_f': 69.8,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'uv': 1.0,
 'gust_mph': 8.9,
 'gust_kph': 14.4}

In [42]:
#nome cidade, pais, temperatura, humidade, pressão do ar e velocidade do vento
results['location']['name']
results['location']['country']
results['current']['temp_c']
results['current']['humidity']
results['current']['pressure_mb']
results['current']['wind_kph']

6.1

In [58]:
results['location'].keys()

dict_keys(['name', 'region', 'country', 'lat', 'lon', 'tz_id', 'localtime_epoch', 'localtime'])

Esta API é muito mais complexa que a API da ISS: não só pela autenticação e parametrização, mas também pela complexidade dos resultados retornados.

Primeiro, vamos construir uma aplicação simples cujo objetivo é guardar apenas as informações temperatura, humidade, pressão do ar e velocidade do vento. Para isso precisamos investigar o resultado retornado e determinar quais chaves iremos guardar:

In [43]:
# EXERCICIO
# Extrair as informações de temperatura, humidade, pressão do ar e velocidade do vento 
# para a lista de latitudes e longitudes abaixo.
lista_latlong = [
    '-23.57,-46.69', 
    '-1.34,-48.60', 
    '-7.99,-34.87', 
    '6.51,3.39',
    '33.69,-84.44', 
    '69.00, 33.13'
]
nome = []
pais = []
temp = []
hum = []
press = []
vel = []

endpoint = 'http://api.weatherapi.com/v1/current.json'
token = 'bfef1b47e3b8462199b191725221801'
header = {'key' : token}


for lat_long in lista_latlong:
    query = {'q' : lat_long}
    response = requests.get(endpoint, headers=header, params=query)
    results = response.json()
    
    nome.append(results['location']['name'])
    pais.append(results['location']['country'])
    temp.append(results['current']['temp_c'])
    hum.append(results['current']['humidity'])
    press.append(results['current']['pressure_mb'])
    vel.append(results['current']['wind_kph'])    

In [46]:
temp

[19.0, 28.0, 26.0, 24.0, 27.8, 18.0]

In [48]:
pd.DataFrame({'name' : nome, 'pais' : pais, 'temp' : temp, 'hum' : hum, 'press' : press, 'vel' : vel})

,name,pais,temp,hum,press,vel
0,Lapa,Brazil,19.0,68,1016.0,11.2
1,Pocao,Brazil,28.0,84,1011.0,13.0
2,Olinda,Brazil,26.0,65,1015.0,22.0
3,Tatala,Nigeria,24.0,89,1015.0,3.6
4,East Point,United States of America,27.8,58,1013.0,9.0
5,Rosta,Russia,18.0,77,1015.0,4.0


### Utilizando `json_normalize()`

No exercicio acima, guardamos apenas um sub-conjunto das informações extraídas. No entanto, muitas vezes é mais prático guardar todas as informações para depois filtra-la.

Para simplificar este processo podemos utilizar a função `json_normalize()` da biblioteca Pandas.

In [49]:
# Verificar variável response.json()
token = 'bfef1b47e3b8462199b191725221801'

endpoint = 'http://api.weatherapi.com/v1/current.json'
header = {'key' : token}
query = {'q' : '-21.99,-47.42'}


response = requests.request('GET', endpoint, headers=header, params=query)
results = response.json()
print(results)

{'location': {'name': 'Piracununga', 'region': 'Sao Paulo', 'country': 'Brazil', 'lat': -21.99, 'lon': -47.42, 'tz_id': 'America/Sao_Paulo', 'localtime_epoch': 1660693282, 'localtime': '2022-08-16 20:41'}, 'current': {'last_updated_epoch': 1660692600, 'last_updated': '2022-08-16 20:30', 'temp_c': 21.0, 'temp_f': 69.8, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 300, 'wind_dir': 'WNW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 56, 'cloud': 0, 'feelslike_c': 21.0, 'feelslike_f': 69.8, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 8.9, 'gust_kph': 14.4}}


Primeiro vamos construir um loop para percorrer uma lista de latitudes e longitudes e armazenar os dicionários de resultados em uma lista:

In [50]:
lista_latlong = [
    '-23.57,-46.69', 
    '-1.34,-48.60', 
    '-7.99,-34.87', 
    '6.51,3.39',
    '33.69,-84.44', 
    '69.00, 33.13'
]

In [51]:
token = 'bfef1b47e3b8462199b191725221801'
endpoint = 'http://api.weatherapi.com/v1/current.json'
header = {'key' : token}
lista_resultados = []

for latlong in lista_latlong:
    query = {'q' : latlong}
    response = requests.request('GET', endpoint, headers=header, params=query)
    results = response.json()
    lista_resultados.append(results)

In [56]:
results

{'location': {'name': 'Rosta',
  'region': 'Murmansk',
  'country': 'Russia',
  'lat': 69.0,
  'lon': 33.13,
  'tz_id': 'Europe/Moscow',
  'localtime_epoch': 1660693307,
  'localtime': '2022-08-17 2:41'},
 'current': {'last_updated_epoch': 1660692600,
  'last_updated': '2022-08-17 02:30',
  'temp_c': 18.0,
  'temp_f': 64.4,
  'is_day': 0,
  'condition': {'text': 'Clear',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
   'code': 1000},
  'wind_mph': 2.5,
  'wind_kph': 4.0,
  'wind_degree': 180,
  'wind_dir': 'S',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 77,
  'cloud': 0,
  'feelslike_c': 18.0,
  'feelslike_f': 64.4,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 17.4,
  'gust_kph': 28.1}}

In [54]:
[type(resultado) for resultado in lista_resultados]

[dict, dict, dict, dict, dict, dict]

A variável `lista_resultados` é uma `list` de `dicts`: cada elemento desta lista contém as informações de uma localização específica em um dicionário.

Agora, vamos utilizar a função `json_normalize()` da biblioteca `Pandas` para transformar esta lista em um `DataFrame`, onde cada linha será correspondente à cada elemento da lista:

In [55]:
tb_weather = pd.json_normalize(lista_resultados)
tb_weather


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,...,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Lapa,Sao Paulo,Brazil,-23.57,-46.69,America/Sao_Paulo,1660693305,2022-08-16 20:41,1660692600,2022-08-16 20:30,...,0.0,68,0,19.0,66.2,10.0,6.0,1.0,1.8,2.9
1,Pocao,Para,Brazil,-1.34,-48.60,America/Belem,1660693305,2022-08-16 20:41,1660692600,2022-08-16 20:30,...,0.0,84,0,30.9,87.5,10.0,6.0,1.0,10.7,17.3
2,Olinda,Pernambuco,Brazil,-7.99,-34.87,America/Recife,1660693305,2022-08-16 20:41,1660692600,2022-08-16 20:30,...,0.0,65,25,27.9,82.1,10.0,6.0,1.0,15.4,24.8
3,Tatala,Lagos,Nigeria,6.51,3.39,Africa/Lagos,1660693306,2022-08-17 0:41,1660692600,2022-08-17 00:30,...,0.0,89,25,25.9,78.6,10.0,6.0,1.0,7.8,12.6
4,East Point,Georgia,United States of America,33.69,-84.44,America/New_York,1660693306,2022-08-16 19:41,1660692600,2022-08-16 19:30,...,0.0,58,0,27.2,80.9,16.0,9.0,8.0,10.3,16.6
5,Rosta,Murmansk,Russia,69.00,33.13,Europe/Moscow,1660693307,2022-08-17 2:41,1660692600,2022-08-17 02:30,...,0.0,77,0,18.0,64.4,10.0,6.0,1.0,17.4,28.1


A função transformou os dicionários hierarquicos em colunas individuais: para evitar a colisão de nomes de colunas, utilizou-se a notação de `.`: por exemplo, o sub-dicionário `location`, que contém campos relativos à localização, foi armazenado nas colunas `location.name`, `location.region`, etc...

In [ ]:
tb_weather.columns

In [ ]:
tb_weather.describe()

# Encapsulando APIs 

Muitas vezes as funções de uma API são encapsuladas em um **wrapper**: uma classe que contém todo o código necessário para autenticar e parametrizar queries em uma dada API. Vamos construir um **wrapper** para a nossa tarefa acima.

In [60]:
class WeatherAPI:
    '''
    Wrapper para a API https://www.weatherapi.com/docs/
    '''
    
    def __init__(self, token):
        '''
        Inicializa o wrapper.
        token Str: Token de identificação do usuário da API
        '''
        self.header = {'key' : token}

        
    def query_latlong(self, lat, long):
        '''
        Busca dados meterológicos atuais para uma localização específica
        lat Float: latitude da localização
        long Float: longitude da localização
        '''
        query = {'q' : f"{lat},{long}"}
        try:
            response = requests.get(endpoint, headers=self.header, params=query)
            response.raise_for_status()
            return response.json()
        except HTTPError as e:
            print(e)
            return None
    

In [61]:
minha_api = WeatherAPI('bfef1b47e3b8462199b191725221801')

In [63]:
minha_api.query_latlong(-15, -47)

{'location': {'name': 'Odilon Guimaraes',
  'region': 'Minas Gerais',
  'country': 'Brazil',
  'lat': -15.0,
  'lon': -47.0,
  'tz_id': 'America/Sao_Paulo',
  'localtime_epoch': 1660694077,
  'localtime': '2022-08-16 20:54'},
 'current': {'last_updated_epoch': 1660693500,
  'last_updated': '2022-08-16 20:45',
  'temp_c': 24.1,
  'temp_f': 75.4,
  'is_day': 0,
  'condition': {'text': 'Clear',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
   'code': 1000},
  'wind_mph': 2.7,
  'wind_kph': 4.3,
  'wind_degree': 179,
  'wind_dir': 'S',
  'pressure_mb': 1014.0,
  'pressure_in': 29.93,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 32,
  'cloud': 11,
  'feelslike_c': 24.4,
  'feelslike_f': 75.9,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 5.6,
  'gust_kph': 9.0}}

# Voltamos 21h11